### Imports

In [ ]:
% matplotlib inline

import sys
import os
os.chdir('../..')
sys.path.append('../..')

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io as io
from IPython.display import Image
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist

from pathlib import Path

import evaluate_catface as evaluate

from project.utils import labeller_fn

# Perform Evaluation

How well do the models perform on your test data?

#### Where are the models located?
Which models do you want to evaluate? Please supply the parent directory of the model directories.

In [ ]:
# Where are the models located?
models_dir = 'models/catface/2hg/2hg-noAug'

filename = Path(models_dir) / 'eval-train80'

# Where are the .tfrecords?
test_data = 'data/tfrecords/catface/80-10-10/train80.tfrecords'

#### Are you ready?
Run the next cell to evaluate the model.
Evaluation metrics returned in a Python dictionary.

In [ ]:
import project.hourglass.params as hgparams

params = {
    hgparams.N_FEATURES: 256,
    hgparams.N_HOURGLASS: 2,
    hgparams.N_RESIDUALS: 3,
}

In [ ]:
def get_immediate_subdirectories(models_dir):
    return [name for name in os.listdir(models_dir)
            if os.path.isdir(os.path.join(models_dir, name))]


print('Will perform evaluation on the following models in `{}`'.format(models_dir))
print(get_immediate_subdirectories(models_dir))
print()


metrics_for_everybody = {}

for name in get_immediate_subdirectories(models_dir):
    model_dir = os.path.join(models_dir, name)
    metrics = evaluate.evaluate(model_dir, test_data, params)
    metrics_for_everybody[name] = metrics
    
for model_dir, metrics in metrics_for_everybody.items():
    print('==========={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {}'.format(metric, value))

## The results are in!

In [ ]:
for model_dir, metrics in metrics_for_everybody.items():
    print('==={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {} <type is `{}`>'.format(metric, value, type(value)))

#### Save to .json

In [ ]:
# import json
# import numpy as np

# class MyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         else:
#             return super(MyEncoder, self).default(obj)
        
# def write_json(filename, model_metrics):
#     print('Writing JSON results to {}'.format(filename))
#     with open(filename, 'w') as handle:
#       json.dump(metrics_for_everybody, handle, sort_keys=True, indent=4, cls=MyEncoder)
    
# if metrics_for_everybody:
#     write_json('{}.json'.format(filename), metrics_for_everybody)

#### Save to .csv

In [ ]:
import pandas as pd

def write_csv(filename, model_metrics):
    print('Writing CSV results to {}'.format(filename))
    df = pd.DataFrame(metrics_for_everybody)
    df.to_csv(filename)
    
if metrics_for_everybody:
    
    print(os.getcwd())
    
    write_csv('{}.csv'.format(filename), metrics_for_everybody)

#### Restore from .csv

In [ ]:
def csv_to_dict(filename):
    dic = pd.DataFrame.from_csv(filename).to_dict()
    return dic

dic = csv_to_dict('{}.csv'.format(filename))

for model_dir, metrics in dic.items():
    print('==========={}========'.format(model_dir))
    for metric, value in metrics.items():
        print('{} = {}'.format(metric, value))